In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [103]:
PATH = 'file:///D:/MSIB/nyc-rolling-sales.csv'
df = pd.read_csv(PATH)

In [104]:
df.shape

(84548, 22)

In [105]:
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


### Merapihkan data frame

In [106]:
from collections import Counter

# mengganti nilai col untuk melihat nilai unik yang muncul di kolom tersebut
col = 'TAX CLASS AT PRESENT'
show = sorted(df[col].dropna().unique())

# show

In [107]:
# convert to ints

df['SALE PRICE'] = [int(price) if price.isdigit() else -1 for price in df['SALE PRICE']]
df['LAND SQUARE FEET'] = [int(size) if size.isdigit() else -1 for size in df['LAND SQUARE FEET']]
df['GROSS SQUARE FEET'] = [int(size) if size.isdigit() else -1 for size in df['GROSS SQUARE FEET']]

In [108]:
# Drop cell
df = df.drop(['Unnamed: 0', 'NEIGHBORHOOD', 'ADDRESS', 'BLOCK', 'LOT', 'EASE-MENT', 'APARTMENT NUMBER'], axis=1)

In [109]:
# add kolom
months = ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df['YEAR SOLD'] = [int(n[0:4]) for n in df['SALE DATE']]
df['MONTH SOLD'] = [int(n[5:7]) for n in df['SALE DATE']]
df['MONTH AND YEAR'] = [months[month - 1] + ' ' + str(year - 2000) for month, year in zip(df['MONTH SOLD'], df['YEAR SOLD'])]

df['PRICE / SQFT'] = [price / sqft if price > -1 and sqft > 0 else -1 for price, sqft in zip(df['SALE PRICE'] , df['GROSS SQUARE FEET'])]
sorted(df['MONTH SOLD'].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [110]:
# melihat data setelah perubahan
df.head()

,BOROUGH,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR SOLD,MONTH SOLD,MONTH AND YEAR,PRICE / SQFT
0,1,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00,2017,7,Jul 17,1028.726708
1,1,07 RENTALS - WALKUP APARTMENTS,2,C7,10009,28,3,31,4616,18690,1900,2,C7,-1,2016-12-14 00:00:00,2016,12,Dec 16,-1.000000
2,1,07 RENTALS - WALKUP APARTMENTS,2,C7,10009,16,1,17,2212,7803,1900,2,C7,-1,2016-12-09 00:00:00,2016,12,Dec 16,-1.000000
3,1,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00,2016,9,Sep 16,579.374742
4,1,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00,2016,11,Nov 16,1733.477790


### MEAN

Tujuan dari program ini adalah untuk membuat ringkasan jumlah penjualan pada tahun 2016 dan 2017 beserta rata-ratanya dalam bentuk DataFrame yang mudah dibaca dan dapat digunakan untuk analisis atau pelaporan lebih lanjut. Dengan menambahkan rata-rata ke DataFrame, program ini memberikan informasi tambahan tentang kinerja penjualan selama dua tahun tersebut.

In [28]:
# Data counts_per_year
counts_per_year = [sum(df['YEAR SOLD'] == 2016), sum(df['YEAR SOLD'] == 2017)]

In [33]:
# Membuat DataFrame dari data
data = {'Tahun': [2016, 2017], 'Jumlah Penjualan': counts_per_year}
df_counts = pd.DataFrame(data)

# Menghitung rata-rata jumlah penjualan
mean_sales = df_counts['Jumlah Penjualan'].mean()

# Menambahkan rata-rata ke DataFrame
df_counts.loc[len(df_counts)] = ['Rata-rata', mean_sales]

print(df_counts)

       Tahun  Jumlah Penjualan
0       2016           29493.0
1       2017           55055.0
2  Rata-rata           42274.0


### Median

Tujuan dari analisis ini adalah untuk mendapatkan nilai median untuk masing-masing kolom unit (RESIDENTIAL UNITS, COMMERCIAL UNITS, dan TOTAL UNITS) berdasarkan kategori BUILDING CLASS CATEGORY. Hasil ini memberikan gambaran yang lebih baik tentang nilai tengah atau karakteristik pusat distribusi unit-unit properti dalam berbagai kategori, yang dapat digunakan untuk analisis lebih lanjut, perbandingan, dan pengambilan keputusan terkait dengan jenis properti yang sesuai dengan kebutuhan atau tujuan bisnis.

In [42]:
# Mengelompokkan data berdasarkan kategori 'BUILDING CLASS CATEGORY' dan menghitung median untuk masing-masing kolom
median_by_category = df.groupby('BUILDING CLASS CATEGORY')[['RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS']].median().reset_index()

# Mengganti nama kolom agar lebih deskriptif
median_by_category.columns = ['BUILDING CLASS CATEGORY', 'Median RESIDENTIAL UNITS', 'Median COMMERCIAL UNITS', 'Median TOTAL UNITS']

# Mencetak tabel median berdasarkan kategori
print(median_by_category)


                         BUILDING CLASS CATEGORY  Median RESIDENTIAL UNITS  \
0    01 ONE FAMILY DWELLINGS                                           1.0   
1    02 TWO FAMILY DWELLINGS                                           2.0   
2    03 THREE FAMILY DWELLINGS                                         3.0   
3    04 TAX CLASS 1 CONDOS                                             1.0   
4    05 TAX CLASS 1 VACANT LAND                                        0.0   
5    06 TAX CLASS 1 - OTHER                                            0.0   
6    07 RENTALS - WALKUP APARTMENTS                                    6.0   
7    08 RENTALS - ELEVATOR APARTMENTS                                 48.0   
8    09 COOPS - WALKUP APARTMENTS                                      0.0   
9    10 COOPS - ELEVATOR APARTMENTS                                    0.0   
10   11 SPECIAL CONDO BILLING LOTS                                     0.0   
11  11A CONDO-RENTALS                                           

### Modus

Tujuan dari analisis data ini adalah untuk mendapatkan pemahaman yang lebih baik tentang karakteristik unit-unit properti berdasarkan kategori BUILDING CLASS CATEGORY. Dengan melakukan perhitungan mean (rata-rata) dan modus (nilai yang paling sering muncul) untuk kolom RESIDENTIAL UNITS, COMMERCIAL UNITS, dan TOTAL UNITS,

In [41]:
# Menghitung mean unit yang telah dihitung
unit_sold_by_category = df.groupby('BUILDING CLASS CATEGORY')[['RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS']].mean()

# Menghitung modus untuk setiap kolom dalam setiap kategori
modus_by_category = df.groupby('BUILDING CLASS CATEGORY')[['RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS']].agg(lambda x: x.mode().iloc[0])

# Memberi nama kolom pada DataFrame hasil
modus_by_category.columns = ['Modus RESIDENTIAL UNITS', 'Modus COMMERCIAL UNITS', 'Modus TOTAL UNITS']

# Menggabungkan mean dan modus menjadi satu DataFrame
result = pd.concat([unit_sold_by_category, modus_by_category], axis=1)

# Menyimpan hasil ke file CSV jika diperlukan
# result.to_csv('hasil_modus.csv', index=False)

# Menampilkan hasil
print(result)


                                              RESIDENTIAL UNITS  \
BUILDING CLASS CATEGORY                                           
01 ONE FAMILY DWELLINGS                                1.001097   
02 TWO FAMILY DWELLINGS                                1.999747   
03 THREE FAMILY DWELLINGS                              2.996578   
04 TAX CLASS 1 CONDOS                                  0.983696   
05 TAX CLASS 1 VACANT LAND                             0.184295   
06 TAX CLASS 1 - OTHER                                 0.148352   
07 RENTALS - WALKUP APARTMENTS                        11.303808   
08 RENTALS - ELEVATOR APARTMENTS                      80.641361   
09 COOPS - WALKUP APARTMENTS                           0.432237   
10 COOPS - ELEVATOR APARTMENTS                         0.975508   
11 SPECIAL CONDO BILLING LOTS                          0.000000   
11A CONDO-RENTALS                                     60.418605   
12 CONDOS - WALKUP APARTMENTS                          1.00000

### Range

Analisis ini bertujuan untuk mengetahui penjual tertinggi dan terendah

In [46]:
# Mencari tanggal minimum dan maksimum dari kolom 'SALE DATE'
tanggal_min = df['SALE DATE'].min().strftime('%Y-%m-%d')
tanggal_max = df['SALE DATE'].max().strftime('%Y-%m-%d')

# Membuat DataFrame untuk output
output_data = {'Deskripsi': ['Tanggal Penjualan Terendah', 'Tanggal Penjualan Tertinggi'],
               'Nilai': [tanggal_min, tanggal_max]}
df_output = pd.DataFrame(output_data)

print(df_output)

                     Deskripsi       Nilai
0   Tanggal Penjualan Terendah  2016-09-01
1  Tanggal Penjualan Tertinggi  2017-08-31


### Variance

Studi kasus pada program ini adalah analisis statistik sederhana terhadap data penjualan properti di New York City (NYC) yang dikumpulkan dalam bentuk DataFrame. Dalam studi kasus ini, kita ingin menghitung varians (variance) dari beberapa atribut penjualan properti yang berbeda, seperti harga penjualan (SALE PRICE), luas tanah (LAND SQUARE FEET), dan luas bangunan (GROSS SQUARE FEET), berdasarkan kategori borough (wilayah) tempat properti tersebut terletak.

In [54]:
# Menghitung variance dari kolom 'SALE PRICE'
variance_sale_price = df['SALE PRICE'].var()

# Membuat DataFrame untuk output
output_data = {'Deskripsi': ['Variance SALE PRICE'], 'Nilai': [variance_sale_price]}
df_output = pd.DataFrame(output_data)

print(df_output)

             Deskripsi         Nilai
0  Variance SALE PRICE  1.675000e+10


In [50]:
# Mengelompokkan data berdasarkan kategori 'BOROUGH' dan menghitung variance untuk masing-masing kolom
variance_by_borough = df.groupby('BOROUGH')[['SALE PRICE', 'LAND SQUARE FEET', 'GROSS SQUARE FEET']].var().reset_index()

# Mengganti nama kolom agar lebih deskriptif
variance_by_borough.columns = ['BOROUGH', 'Variance SALE PRICE', 'Variance LAND SQUARE FEET', 'Variance GROSS SQUARE FEET']

# Mencetak tabel variance berdasarkan 'BOROUGH'
print(variance_by_borough)

   BOROUGH  Variance SALE PRICE  Variance LAND SQUARE FEET  \
0        1         4.598839e+14               2.938062e+07   
1        2         7.745411e+12               3.420048e+08   
2        3         1.549063e+13               1.980388e+08   
3        4         7.330386e+12               1.393263e+09   
4        5         2.645131e+12               6.871647e+09   

   Variance GROSS SQUARE FEET  
0                1.424622e+09  
1                8.726635e+08  
2                2.348546e+08  
3                1.157928e+09  
4                1.542479e+08  


Studi kasus ini berguna untuk memahami sejauh mana variasi dalam atribut penjualan properti (harga, luas tanah, dan luas bangunan) berbeda antarwilayah di NYC. Varians adalah salah satu metrik yang membantu kita dalam analisis statistik untuk mengukur sebaran atau variasi data dalam suatu populasi atau sampel. Studi kasus ini dapat memberikan wawasan kepada pemilik properti, pengembang, atau analis real estate mengenai perbedaan dalam pasar properti antarwilayah di NYC.

###  Standard Deviation

In [55]:
# Mengelompokkan data berdasarkan kolom 'BOROUGH'
grouped_by_borough = df.groupby('BOROUGH')['SALE PRICE']

# Menghitung Standard Deviation dari harga penjualan untuk setiap wilayah (BOROUGH)
standard_deviation_by_borough = grouped_by_borough.std().reset_index()

# Mengganti nama kolom agar lebih deskriptif
standard_deviation_by_borough.columns = ['BOROUGH', 'Standard Deviation SALE PRICE']

# Mencetak tabel Standard Deviation berdasarkan wilayah (BOROUGH)
print(standard_deviation_by_borough)

   BOROUGH  Standard Deviation SALE PRICE
0        1                            NaN
1        2                            NaN
2        3                            NaN
3        4                            NaN
4        5                            NaN


In [56]:
# Pilih wilayah Manhattan (BOROUGH 1) sebagai fokus studi kasus
manhattan_data = df[df['BOROUGH'] == 1]

# Menghitung Standard Deviation dari luas bangunan (GROSS SQUARE FEET) di Manhattan
standard_deviation_manhattan = manhattan_data['GROSS SQUARE FEET'].std()

# Mencetak hasil Standard Deviation
print("Standard Deviation Luas Bangunan di Manhattan:", standard_deviation_manhattan)

Standard Deviation Luas Bangunan di Manhattan: nan


tujuan utama dari analisis data ini adalah untuk memberikan pemahaman yang lebih baik tentang sebaran luas bangunan di wilayah Manhattan dalam konteks penjualan properti di New York City. Analisis ini dapat membantu stakeholders dalam mengambil keputusan yang lebih baik terkait properti di wilayah tersebut.

###  Probability Distribution 

Tujuan analisis ini adalah untuk memprediksi probabilitas penjualan properti di wilayah Manhattan, New York City, pada bulan tertentu (Desember 2016) dengan menggunakan distribusi probabilitas binomial. Tujuan utama dari analisis ini adalah untuk memprediksi probabilitas penjualan properti di wilayah Manhattan selama bulan Desember 2016. Dalam hal ini, probabilitas penjualan adalah seberapa besar kemungkinan penjualan properti akan terjadi dalam bulan yang ditentukan.

In [79]:
# Pilih kolom 'BOROUGH' dan 'SALE DATE' dari DataFrame
selected_columns = df[['BOROUGH', 'SALE DATE']]

# Tampilkan tabel yang berisi kolom 'BOROUGH' dan 'SALE DATE'
print(selected_columns)

       BOROUGH            SALE DATE
0            1  2017-07-19 00:00:00
1            1  2016-12-14 00:00:00
2            1  2016-12-09 00:00:00
3            1  2016-09-23 00:00:00
4            1  2016-11-17 00:00:00
...        ...                  ...
84543        5  2016-11-28 00:00:00
84544        5  2017-04-21 00:00:00
84545        5  2017-07-05 00:00:00
84546        5  2016-12-21 00:00:00
84547        5  2016-10-27 00:00:00

[84548 rows x 2 columns]


In [83]:
# Konversi kolom 'SALE DATE' ke tipe data datetime
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

# Filter data untuk wilayah tertentu (misalnya, Manhattan - BOROUGH 1)
manhattan_data = df[df['BOROUGH'] == 1]

# Tentukan tanggal awal dan akhir untuk analisis (misalnya, Januari 2023)
tanggal_awal = pd.to_datetime('2016-12-01')
tanggal_akhir = pd.to_datetime('2016-12-30')

# Filter data berdasarkan tanggal
penjualan_desember = manhattan_data[(manhattan_data['SALE DATE'] >= tanggal_awal) & (manhattan_data['SALE DATE'] <= tanggal_akhir)]

# Menghitung jumlah penjualan properti dalam bulan tertentu (n)
n = len(penjualan_desember)

# Probabilitas penjualan satu properti dalam satu percobaan (p)
# Anda juga dapat mengganti nilai ini berdasarkan data historis
p = 0.05  # Misalnya, probabilitas penjualan adalah 5%

# Menghitung probabilitas penjualan properti di bulan tertentu
probabilitas_penjualan = np.random.binomial(n, p) / n

# Mencetak hasil probabilitas penjualan
print(f'Probabilitas penjualan properti di Manhattan pada bulan Desember 2016: {probabilitas_penjualan:.2%}')

Probabilitas penjualan properti di Manhattan pada bulan Desember 2016: 5.67%


###  Confidence Intervals

Tujuan analisis ini adalah untuk memberikan perkiraan rentang harga penjualan properti di wilayah Manhattan (BOROUGH 1) dalam dataset NYC Property Sales dengan menggunakan Confidence Intervals (Interval Kepercayaan) pada tingkat kepercayaan 95%. Tujuan utama dari analisis ini adalah untuk mengestimasi rentang harga penjualan properti di wilayah Manhattan yang memiliki tingkat kepercayaan 95%. Rentang ini memberikan batasan atas dan batasan bawah dari harga penjualan yang mungkin berlaku di wilayah tersebut.Tingkat kepercayaan sebesar 95% yang digunakan dalam analisis ini berarti kita memiliki 95% kepercayaan bahwa rata-rata harga penjualan properti di Manhattan berada dalam rentang Confidence Intervals yang dihasilkan. Ini memberikan ukuran keyakinan terhadap estimasi harga.

In [84]:
from scipy import stats

# Filter data untuk wilayah Manhattan (BOROUGH 1)
manhattan_data = df[df['BOROUGH'] == 1]

# Mengambil kolom 'SALE PRICE' sebagai data harga penjualan di Manhattan
sale_price_manhattan = manhattan_data['SALE PRICE']

# Menghitung statistik dasar, seperti rata-rata dan standar deviasi
mean_price = sale_price_manhattan.mean()
std_deviation = sale_price_manhattan.std()

# Ukuran sampel (jumlah penjualan di Manhattan)
sample_size = len(sale_price_manhattan)

# Menghitung Confidence Intervals dengan tingkat kepercayaan 95%
confidence_level = 0.95
margin_of_error = stats.norm.ppf((1 + confidence_level) / 2) * (std_deviation / np.sqrt(sample_size))
confidence_interval = (mean_price - margin_of_error, mean_price + margin_of_error)

# Mencetak hasil Confidence Intervals
print(f"Confidence Intervals untuk rata-rata harga penjualan di Manhattan (95% kepercayaan):")
print(f"Rentang: ${confidence_interval[0]:,.2f} - ${confidence_interval[1]:,.2f}")

Confidence Intervals untuk rata-rata harga penjualan di Manhattan (95% kepercayaan):
Rentang: $2,322,181.88 - $2,943,487.79


### Hypothesis Testing

Tujuan dari analisis data ini adalah untuk menentukan apakah terdapat perbedaan yang signifikan antara rata-rata harga penjualan properti di wilayah Manhattan (BOROUGH 1) dan Brooklyn (BOROUGH 3) dalam dataset NYC Property Sales. 


In [85]:
import scipy.stats as stats

# Filter data untuk wilayah Manhattan (BOROUGH 1) dan Brooklyn (BOROUGH 3)
manhattan_data = df[df['BOROUGH'] == 1]['SALE PRICE']
brooklyn_data = df[df['BOROUGH'] == 3]['SALE PRICE']

# Melakukan uji t independen untuk membandingkan dua sampel
# Ini menguji hipotesis bahwa rata-rata harga penjualan di Manhattan dan Brooklyn berbeda.
t_stat, p_value = stats.ttest_ind(manhattan_data, brooklyn_data, equal_var=False)

# Tingkat signifikansi (alpha)
alpha = 0.05

# Mencetak hasil uji t
print(f"Nilai t-statistic: {t_stat}")
print(f"Nilai p-value: {p_value}")

# Melakukan pengujian hipotesis dengan tingkat signifikansi alpha
if p_value < alpha:
    print("Hipotesis nol (H0) ditolak: Terdapat perbedaan yang signifikan antara rata-rata harga penjualan di Manhattan dan Brooklyn.")
else:
    print("Hipotesis nol (H0) tidak dapat ditolak: Tidak ada perbedaan yang signifikan antara rata-rata harga penjualan di Manhattan dan Brooklyn.")

Nilai t-statistic: 11.203353864767172
Nilai p-value: 4.8286713687219043e-29
Hipotesis nol (H0) ditolak: Terdapat perbedaan yang signifikan antara rata-rata harga penjualan di Manhattan dan Brooklyn.


Kesimpulan dari analisis Hypothesis Testing pada data penjualan properti di wilayah Manhattan (BOROUGH 1) dan Brooklyn (BOROUGH 3) dalam dataset NYC Property Sales adalah sebagai berikut:

Berdasarkan hasil uji t independen dengan tingkat signifikansi (alpha) sebesar 0.05, kami menyimpulkan:

- Nilai t-statistic adalah [nilai t-statistic yang dihasilkan].
- Nilai p-value adalah [nilai p-value yang dihasilkan].

Dengan demikian, kita membuat kesimpulan sebagai berikut:

**Jika p-value < alpha (0.05)**:
- **Kesimpulan**: Hipotesis nol (H0) ditolak.
- **Interpretasi**: Ada perbedaan yang signifikan antara rata-rata harga penjualan properti di wilayah Manhattan (BOROUGH 1) dan Brooklyn (BOROUGH 3).

**Jika p-value >= alpha (0.05)**:
- **Kesimpulan**: Hipotesis nol (H0) tidak dapat ditolak.
- **Interpretasi**: Tidak ada perbedaan yang signifikan antara rata-rata harga penjualan properti di wilayah Manhattan dan Brooklyn.

Kesimpulan ini didasarkan pada tingkat signifikansi yang telah ditentukan. Dalam contoh ini, dengan tingkat signifikansi 0.05, kita menyimpulkan bahwa terdapat perbedaan yang signifikan antara harga penjualan properti di Manhattan dan Brooklyn.

Kesimpulan ini dapat membantu dalam pengambilan keputusan terkait investasi dan bisnis properti di kedua wilayah tersebut.

### Preprocessing

In [111]:
# 1. Mengatasi Data yang Hilang
mean_sale_price = df['SALE PRICE'].mean()
df['SALE PRICE'].fillna(mean_sale_price, inplace=True)

# 2. Pengkodean Data Kategori
df = pd.get_dummies(df, columns=['BOROUGH'], prefix='BOROUGH')

# 3. Normalisasi
min_land_square_feet = df['LAND SQUARE FEET'].min()
max_land_square_feet = df['LAND SQUARE FEET'].max()
df['LAND SQUARE FEET'] = (df['LAND SQUARE FEET'] - min_land_square_feet) / (max_land_square_feet - min_land_square_feet)

# 4. Penyederhanaan Data
selected_columns = ['SALE PRICE', 'BOROUGH_1', 'BOROUGH_2', 'BOROUGH_3', 'BOROUGH_4', 'BOROUGH_5', 'LAND SQUARE FEET']
df = df[selected_columns]

# 5. Penyimpanan Data (opsional)
# Data yang telah diproses dapat disimpan dalam format yang sesuai, seperti CSV, untuk analisis lebih lanjut.
#df.to_csv('nyc_property_sales_preprocessed.csv', index=False)


In [112]:
# Menampilkan beberapa baris pertama data yang telah diolah
print(df.head())

   SALE PRICE  BOROUGH_1  BOROUGH_2  BOROUGH_3  BOROUGH_4  BOROUGH_5  \
0     6625000          1          0          0          0          0   
1          -1          1          0          0          0          0   
2          -1          1          0          0          0          0   
3     3936272          1          0          0          0          0   
4     8000000          1          0          0          0          0   

   LAND SQUARE FEET  
0          0.000384  
1          0.001086  
2          0.000520  
3          0.000535  
4          0.000557  


Tujuan dari analisis data di atas, yang mencakup berbagai tahapan pra-pemrosesan data, adalah untuk mempersiapkan data penjualan properti di NYC sehingga dapat digunakan secara efektif dalam analisis lebih lanjut. Beberapa tujuan utama dari analisis ini adalah:

1. **Membersihkan Data**: Mengatasi masalah data yang hilang, seperti mengisi nilai yang hilang dalam kolom SALE PRICE dengan rata-rata harga penjualan. Hal ini dilakukan agar tidak ada data yang hilang yang dapat mengganggu analisis.

2. **Pengkodean Data Kategori**: Menggunakan teknik one-hot encoding untuk kolom BOROUGH, yang merupakan data kategori. Ini memungkinkan penggunaan data kategori dalam analisis statistik.

3. **Normalisasi Data**: Melakukan normalisasi Min-Max pada kolom LAND SQUARE FEET agar data memiliki skala yang seragam. Ini membantu dalam menghindari efek skala yang berlebihan dalam analisis.

4. **Penyederhanaan Data**: Memilih kolom-kolom yang relevan untuk analisis, seperti SALE PRICE, BOROUGH, dan LAND SQUARE FEET. Ini membantu dalam fokus pada variabel-variabel yang penting.

5. **Penyimpanan Data (Opsional)**: Data yang telah diproses dapat disimpan dalam format yang sesuai, seperti CSV. Ini memungkinkan penggunaan data yang sudah dipersiapkan dalam analisis selanjutnya tanpa perlu melakukan pra-pemrosesan ulang.

Tujuan akhir dari semua langkah pra-pemrosesan ini adalah untuk membuat data yang lebih bersih, terstruktur, dan siap digunakan dalam analisis harga properti di NYC. Data yang telah dipersiapkan dengan baik akan memberikan dasar yang kuat untuk analisis lebih lanjut, seperti perbandingan harga properti di berbagai wilayah atau peramalan harga properti di masa depan.